In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import euclidean_distances, cosine_distances

# import and run file
from importnb import imports
with imports("ipynb"):
    import data_preparation
%run data_preparation.ipynb

In [ ]:
# one-hot encoding for categorical
data_cat = restaurants[['Cuisine', 'Open After 8pm?']]
data_cat = pd.get_dummies(data_cat, columns=['Cuisine', 'Open After 8pm?'], drop_first=True, dtype=int)
# standardization for numerical
data_num = restaurants[['Latitude', 'Longitude', 'Average Cost']]
scaler = StandardScaler()
data_num = scaler.fit_transform(data_num)

data_restaurant = np.column_stack((data_num, data_cat))

In [ ]:
# euclidean distance
restaurants_euclidean = pd.DataFrame(
    euclidean_distances(data_restaurant, data_restaurant),
    columns=restaurants['Restaurant Name'],
    index=restaurants['Restaurant Name']
)
# cosine distance
restaurants_cosine = pd.DataFrame(
    cosine_distances(data_restaurant, data_restaurant),
    columns=restaurants['Restaurant Name'],
    index=restaurants['Restaurant Name']
)

In [ ]:
def contentfilter_recommendation(name, score_type='Euclidean'):
    reviewer_restaurants = reviews[reviews['Reviewer Name'] == name].sort_values(['Rating'], ascending=False)
    fav_restaurant = reviewer_restaurants['Restaurant Name'].iloc[0]
    if (score_type == 'Cosine'):
        data_ret = restaurants_cosine.loc[:,fav_restaurant].sort_values(ascending=True)
        return( data_ret[data_ret.index != fav_restaurant].head(10) )
    else:
        data_ret = restaurants_euclidean.loc[:,fav_restaurant].sort_values(ascending=True)
        return( data_ret[data_ret.index != fav_restaurant].head(10) )

In [ ]:
print(contentfilter_recommendation('Willie Jacobsen'))

In [ ]:
print(contentfilter_recommendation('Willie Jacobsen', 'Cosine'))